## Es wird Zeit anzufangen...

Die meisten KI-Anbieter bieten eine eigene API zum Kommunizieren an. Diese sind meist ziemlich "low-level", d.h. manche Komfort-Funktion muss man selber entwickeln. Die APIs sind häufig vom Anbieter zu Anbieter ähnlich, aber nicht kompatibel.
In den meisten Fällen handelt es sich um REST-Endpunkte und JSON als Datenprotokoll. Eigentlich immer muss man einen API-Key mitgeben, den man bei dem Anbieter bekommt. Die Nutzung kostet dann pro verbrauchter Token.

Starten wir mit einer LangChain-Einführung. 

### Was ist LangChain?
LangChain ist ein Framework um programmatisch mit KIs zu kommunizieren. In seiner einfachsten Form ist es eine einheitliche API für alle möglichen Sprachmodelle. Zusätzlich bietet es zahlreiche Utilities und Helferlein für alle erdenklichen Aufgaben rund um das Thema KI. Der Vorteil bei der Nutzung von solchen Frameworks ist, dass man sich weniger stark an den Anbieter von dem jeweiligen KI-Modells bindet, die ja meist ihre eigenen APIs anbieten. So kann man sehr leicht bestehenden Code auf ein anderes Modell jagen, ohne dabei viel anpassen zu müssen.

LangChain ist eines der am stärksten verbreiteten Frameworks. Das hat sicher damit zu tun, dass LangChain relativ früh am Anfang des Hypes, der nach dem Launch von ChatGPT losgeganen ist, zur Verfügung stand. Wegen der Verbreitung und der unzähligen Beispiele, die man im Internet findet, habe ich mich für die Nutzung dieses Frameworks für diesen Workshop entschieden. Allerdings ist es meines Erachten nach, nicht immer das beste Framework. Oft gibt es für bestimmte Anwendungsfälle besser geeigente Frameworks, wie z.B. LlamaIndex. 
Meine Kritikpunkte an LangChain sind: das Framework ist häufig inkonsistent und so hat man manchmal Situationen, wo man unterschiedliche Herangehensweisen für unterschiedliche Aufgaben benötigt, auch wenn man nur das Framework nutzt.

Das Framework hat sich auch in den letzten Monaten stark verändert und so ist es manchmal etwas mühsam Beispielcode aus dem Internet wieder zum Laufen zu bringen, da vieles anders gelöst wird. Bitte seht es mir nach, wenn ich hier und da auch ins Straucheln komme! :)

Wenn du dich selber ein wenig Schlau machen möchtest, so findest du hier eine Einführungsseite für LangChain: https://python.langchain.com/v0.2/docs/introduction/




## Wie ist LangChain aufgebaut?

Sehen wir uns erstmal eine Grafik an...

![LangChain Stack](https://python.langchain.com/v0.2/svg/langchain_stack_dark.svg)


Wie man sieht, ist LangChain ein Ökosystem aus vielen verschiedenen Tools und Framworks. Es gibt unzählige Community-Umsetzung von Adaptern und Tools und somit kann man LangChain beinahe mit allem Erdenkliche verbinden.

## Übung #1 - Installation und erster Aufruf
Damit wir loslegen können, müssen wir zunächst alles was wir brauchen installieren. In der Python-Welt übernimmt das häufig der Paket-Manager `pip`.
Als erstes brauchen wir natürlich LangChain selbst.

In [ ]:
!pip install langchain langchain_openai python-dotenv

In diese Workshop nutzen wir die Modelle von OpenAI, also GPT-4o z.B.

Damit wir das tun können, müssen wir einen API-Key bekommen. In diesem Workshop werde ich euch einen zur Verfügung stellen.

Den API-Key fügen wir in die .env-Datei, in der Form OPENAI_API_KEY=..., ein.

Lass uns ein erstes super einfaches Programm schreiben.

In [ ]:
import os
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")

# model = ChatOpenAI(model="gpt-4o")
model = AzureChatOpenAI(openai_api_version="2024-05-01-preview", azure_deployment="gpt-4o", temperature=0.5)

messages = [
    SystemMessage("Du bist ein lustiger Chatbot, der auf Fragen antwortet aber sich so verhält als wäre er ein Minion."),
    HumanMessage("Schreibe eine Begrüßung für die Teilnehmer des KI-Agenten-Workshops."),
]

model.invoke(messages)

Das hat hoffentlich bei dir funktioniert und du hast eine mehr oder weniger lustige Antwort bekommen.

Was wir natürlich auch sehen, dass wir neben der Antwort eine Menge weiterer Informationen bekommen. Im Namen des LangChain Frameworks steck ja der Begriff _Chain_ also Kette. Wir werden gleich sehen, warum das so ist.

Das Framework bietet nämlich die Verkettung von Verarbeitungsschritten an, die in einem sehr schlanken und gut lesbaren Code münden.

Nehmen wir an, wir wollen nur die eigentliche Antwort ausgeben. Hierfür bietet LangChain einen _OutputParser_ an. Das ist eins von unzähligen Werkzeugen in diesem Framework.

Der folgende Code zeigt, wie der Parser importiert und angelegt wird und dann in die Verabeitungskette eingefügt wird.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser

chain.invoke(messages)

Jetzt wird es relativ deutlich, wie LangChain "tickt". Durch das "|" Symbol können wir also unterschiedliche Bestandteile zu einer Kette zusammenfügen.

Wir wollen uns bald an die Agenten wagen, aber bevor wir das tun, sollten wir uns ein wichtiges Konzept noch vorher ansehen.

Wenn wir es mit KIs zu tun haben, müssen wir immer Prompts übergeben. In den wenigesten Fällen sind die Prompts fest verdrahtet und sind meist dynamisch gestaltet. Auch hiefür hat LangChain etwas im Petto, nämlich die _PromptTemplates_.

Zwar kann man in Python recht einfach mit den sog. f-Strings Template-Strings sehr gut und einfach mit Werten befüllen aber die PromptTemplates fügen sich in die LangChain-Welt besser ein.

Lass uns das vorherige Beispiel aufbohren und z.B. das Verhalten der KI nicht festverdrahten, sondern per Parameter übergeben.

_(Das Beispiel ist eigentlich ein Musterbeispiel dafür, warum diese API manchmal etwas irritierend ist... mehr dazu auf der Tonspur...)_

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Du bist ein lustiger Chatbot, der auf Fragen antwortet aber sich so verhält als wäre er {person}."

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

# Direkte Ausführung des Templates als Beispiel, wie das Template funktioniert (Achtung: das hier ist kein Aufruf des KI... wir sezten nur das Template ein, um zu zeigen, wie es funktioniert.)
result = prompt_template.invoke({"person": "ein verrückter Professor", "text": "Schreibe eine Begrüßung für die Teilnehmer des KI-Agenten-Workshops."})

result

Jetzt fügen wir das Ganze zu einer Kette zusammen.

In [ ]:
chain = prompt_template | model | parser

chain.invoke({"person": "ein verrückter Professor", "text": "Schreibe eine Begrüßung für die Teilnehmer des KI-Agenten-Workshops."})